# Add VG mail status to precincts
- Start with Field_map_ETL.ipnb output file
    - bcdp/field_map_ETL/output/pct_area_boulder.geojson
- Read VG mail status from GOTV files
- Merge Mountain and Rural column

In [28]:
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np

In [16]:
pct = gpd.read_file('output/pct_area_boulder.geojson')

In [17]:
pct

,index,DISTRICT,precinct,area_short,area_long,geometry
0,509,2181207509,509,GN-01,Gunbarrel-Niwot-01,"POLYGON ((-105.15943 40.10160, -105.15942 40.1..."
1,700,2151207700,700,MT-03,Mountains-03,"POLYGON ((-105.13106 40.12712, -105.13105 40.1..."
2,702,2154907702,702,MT-03,Mountains-03,"POLYGON ((-105.15906 40.26122, -105.15906 40.2..."
3,701,2154907701,701,MT-03,Mountains-03,"POLYGON ((-105.23753 40.21215, -105.23759 40.2..."
4,916,2154907916,916,MT-03,Mountains-03,"POLYGON ((-105.19594 40.26146, -105.19971 40.2..."
...,...,...,...,...,...,...
188,904,2154907904,904,MT-01,Mountains-01,"POLYGON ((-105.50867 39.97230, -105.50952 39.9..."
189,903,2154907903,903,MT-01,Mountains-01,"POLYGON ((-105.51588 40.02758, -105.51589 40.0..."
190,002,2151907002,002,LM-04,Longmont-04,"POLYGON ((-105.05519 40.19115, -105.05518 40.1..."
191,003,2184907003,003,SU-01,Superior-01,"POLYGON ((-105.16920 39.92870, -105.17207 39.9..."


## Read in mail data

In [18]:
pct_mail = pd.read_csv('../Field/data/pct22_mail_final.csv', dtype=str)

In [19]:
pct_mail.columns

Index(['precinct', 'area_short', 'area_long', 'district', 'district.1', 'CD',
       'HD', 'SD', 'pct21_raw', 'pct21', 'rural'],
      dtype='object')

In [20]:
pct_mail = pct_mail[['precinct', 'rural']]

In [21]:
pct_mail

,precinct,rural
0,901,x
1,902,x
2,903,x
3,904,NaN
4,905,x
...,...,...
188,910,x
189,914,x
190,913,x
191,912,x


In [29]:
pct_mail['mail'] = pct_mail.rural.map({'x':'mail'})

In [30]:
pct_mail

,precinct,rural,mail
0,901,x,mail
1,902,x,mail
2,903,x,mail
3,904,NaN,NaN
4,905,x,mail
...,...,...,...
188,910,x,mail
189,914,x,mail
190,913,x,mail
191,912,x,mail


In [33]:
pct_mail.mail.fillna('walk', inplace=True)

In [34]:
pct_mail.value_counts('mail')

mail
walk    165
mail     28
dtype: int64

In [35]:
pct = pct.merge(pct_mail, left_on='precinct', right_on='precinct', how='left')

In [36]:
pct.head()

,index,DISTRICT,precinct,area_short,area_long,geometry,rural,mail
0,509,2181207509,509,GN-01,Gunbarrel-Niwot-01,"POLYGON ((-105.15943 40.10160, -105.15942 40.1...",NaN,walk
1,700,2151207700,700,MT-03,Mountains-03,"POLYGON ((-105.13106 40.12712, -105.13105 40.1...",x,mail
2,702,2154907702,702,MT-03,Mountains-03,"POLYGON ((-105.15906 40.26122, -105.15906 40.2...",x,mail
3,701,2154907701,701,MT-03,Mountains-03,"POLYGON ((-105.23753 40.21215, -105.23759 40.2...",x,mail
4,916,2154907916,916,MT-03,Mountains-03,"POLYGON ((-105.19594 40.26146, -105.19971 40.2...",x,mail


In [37]:
pct.drop(columns=['index'], inplace=True)

In [38]:
pct.head()

,DISTRICT,precinct,area_short,area_long,geometry,rural,mail
0,2181207509,509,GN-01,Gunbarrel-Niwot-01,"POLYGON ((-105.15943 40.10160, -105.15942 40.1...",NaN,walk
1,2151207700,700,MT-03,Mountains-03,"POLYGON ((-105.13106 40.12712, -105.13105 40.1...",x,mail
2,2154907702,702,MT-03,Mountains-03,"POLYGON ((-105.15906 40.26122, -105.15906 40.2...",x,mail
3,2154907701,701,MT-03,Mountains-03,"POLYGON ((-105.23753 40.21215, -105.23759 40.2...",x,mail
4,2154907916,916,MT-03,Mountains-03,"POLYGON ((-105.19594 40.26146, -105.19971 40.2...",x,mail


In [39]:
pct.to_file('output/pct_area_boulder_mail.geojson', driver='GeoJSON')

C:\Users\BruceDev\miniconda3\envs\geo_ds\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
